### Генерация поэзии с помощью нейронных сетей: шаг 1
##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

Ваша основная задача: научиться генерироват стихи с помощью простой рекуррентной нейронной сети (Vanilla RNN). В качестве корпуса текстов для обучения будет выступать роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина.

In [8]:
# do not change the code in the block below
# __________start of block__________
import string
import os
from random import sample

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from IPython.display import clear_output

import matplotlib.pyplot as plt
# __________end of block__________

In [3]:
# do not change the code in the block below
# __________start of block__________
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('{} device is available'.format(device))
# __________end of block__________

cuda device is available


#### 1. Загрузка данных.

In [5]:
# do not change the code in the block below
# __________start of block__________
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt
    
with open('onegin.txt', 'r') as iofile:
    text = iofile.readlines()
    
text = "".join([x.replace('\t\t', '').lower() for x in text])
# __________end of block__________

--2024-12-07 16:05:57--  https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262521 (256K) [text/plain]
Saving to: 'onegin.txt'

onegin.txt          100%[===================>] 256.37K  --.-KB/s    in 0.03s   

2024-12-07 16:05:57 (9.38 MB/s) - 'onegin.txt' saved [262521/262521]



#### 2. Построение словаря и предобработка текста
В данном задании требуется построить языковую модель на уровне символов. Приведем весь текст к нижнему регистру и построим словарь из всех символов в доступном корпусе текстов. Также добавим токен `<sos>`.

In [6]:
# do not change the code in the block below
# __________start of block__________
tokens = sorted(set(text.lower())) + ['<sos>']
num_tokens = len(tokens)

assert num_tokens == 84, "Check the tokenization process"

token_to_idx = {x: idx for idx, x in enumerate(tokens)}
idx_to_token = {idx: x for idx, x in enumerate(tokens)}

assert len(tokens) == len(token_to_idx), "Mapping should be unique"

print("Seems fine!")


text_encoded = [token_to_idx[x] for x in text]
# __________end of block__________

Seems fine!


__Ваша задача__: обучить классическую рекуррентную нейронную сеть (Vanilla RNN) предсказывать следующий символ на полученном корпусе текстов и сгенерировать последовательность длины 100 для фиксированной начальной фразы.

Вы можете воспользоваться кодом с занятие №6 или же обратиться к следующим ссылкам:
* Замечательная статья за авторством Andrej Karpathy об использовании RNN: [link](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* Пример char-rnn от Andrej Karpathy: [github repo](https://github.com/karpathy/char-rnn)
* Замечательный пример генерации поэзии Шекспира: [github repo](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb)

Данное задание является достаточно творческим. Не страшно, если поначалу оно вызывает затруднения. Последняя ссылка в списке выше может быть особенно полезна в данном случае.

Далее для вашего удобства реализована функция, которая генерирует случайный батч размера `batch_size` из строк длиной `seq_length`. Вы можете использовать его при обучении модели.

In [10]:
# do not change the code in the block below
# __________start of block__________
batch_size = 256
seq_length = 100
start_column = np.zeros((batch_size, 1), dtype=int) + token_to_idx['<sos>']

def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size*seq_length - 1)
    data = np.array(text_encoded[start_index:start_index + batch_size*seq_length]).reshape((batch_size, -1))
    yield np.hstack((start_column, data))
# __________end of block__________    

Пример батча:

In [11]:
next(generate_chunk())

array([[83, 64, 61, ..., 58, 50, 63],
       [83,  5,  1, ...,  1, 45, 61],
       [83, 57, 53, ..., 61, 52, 45],
       ...,
       [83, 45, 57, ..., 51,  5,  1],
       [83, 59, 58, ..., 47,  1, 60],
       [83, 59, 56, ..., 58, 73,  1]])

Далее вам предстоит написать код для обучения модели и генерации текста.

In [61]:
# your beautiful experiments here

# Параметры модели
vocab_size = len(idx_to_token)        # Размер словаря
embed_size = 128                      # Размер эмбеддингов
hidden_size = 256                     # Размер скрытого слоя
num_layers = 1                        # Количество слоев RNN

# Гиперпараметры обучения
num_epochs = 100                       # Количество эпох
learning_rate = 0.002                 # Скорость обучения
clip_grad = 5.0                       # Ограничение градиентов для предотвращения исчезновения или взрыва градиентов

In [62]:
class VanillaRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super(VanillaRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Векторизация входных символов
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # Рекуррентный слой
        self.rnn = nn.RNN(embed_size, hidden_size, num_layers, batch_first=True)

        # Полносвязный слой для предсказания следующего символа
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        """
        x: (batch_size, seq_length)
        hidden: (num_layers, batch_size, hidden_size)
        """
        embedded = self.embedding(x)  # (batch_size, seq_length, embed_size)
        out, hidden = self.rnn(embedded, hidden)  # out: (batch_size, seq_length, hidden_size)
        out = out.contiguous().view(-1, self.hidden_size)  # (batch_size * seq_length, hidden_size)
        out = self.fc(out)  # (batch_size * seq_length, vocab_size)
        return out, hidden

    def init_hidden(self, batch_size):
        # Инициализация скрытого состояния нулями
        return torch.zeros(self.num_layers, batch_size, self.hidden_size)

In [63]:
# Инициализация модели
model = VanillaRNN(vocab_size, embed_size, hidden_size, num_layers).to(device)

# Функция потерь
criterion = nn.CrossEntropyLoss()

# Оптимизатор
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [64]:
model.train()  # Переводим модель в режим обучения

for epoch in range(1, num_epochs + 1):
    # Генерация батча данных
    batch = next(generate_chunk())  # (batch_size, seq_length + 1)
    inputs = batch[:, :-1]         # Входные символы
    targets = batch[:, 1:].reshape(-1)  # Целевые символы (сдвинуто на один вперед)

    # Перенос данных на устройство
    inputs = torch.tensor(inputs, dtype=torch.long).to(device)
    targets = torch.tensor(targets, dtype=torch.long).to(device)

    # Инициализация скрытого состояния
    hidden = model.init_hidden(batch_size).to(device)

    # Обнуление градиентов
    optimizer.zero_grad()

    # Прямой проход
    outputs, hidden = model(inputs, hidden.detach())

    # Вычисление потерь
    loss = criterion(outputs, targets)

    # Обратный проход
    loss.backward()

    # Ограничение градиентов
    nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

    # Обновление параметров
    optimizer.step()

    # Печать информации об обучении
    print(f"Эпоха [{epoch}/{num_epochs}], Потери: {loss.item():.4f}")

Эпоха [1/100], Потери: 4.4732
Эпоха [2/100], Потери: 4.2402
Эпоха [3/100], Потери: 3.9724
Эпоха [4/100], Потери: 3.6177
Эпоха [5/100], Потери: 3.3431
Эпоха [6/100], Потери: 3.2370
Эпоха [7/100], Потери: 3.1704
Эпоха [8/100], Потери: 3.1072
Эпоха [9/100], Потери: 3.0590
Эпоха [10/100], Потери: 3.0213
Эпоха [11/100], Потери: 2.9795
Эпоха [12/100], Потери: 2.9114
Эпоха [13/100], Потери: 2.8950
Эпоха [14/100], Потери: 2.8733
Эпоха [15/100], Потери: 2.8339
Эпоха [16/100], Потери: 2.8002
Эпоха [17/100], Потери: 2.7786
Эпоха [18/100], Потери: 2.7614
Эпоха [19/100], Потери: 2.7281
Эпоха [20/100], Потери: 2.7388
Эпоха [21/100], Потери: 2.6964
Эпоха [22/100], Потери: 2.7007
Эпоха [23/100], Потери: 2.6731
Эпоха [24/100], Потери: 2.6553
Эпоха [25/100], Потери: 2.6300
Эпоха [26/100], Потери: 2.6103
Эпоха [27/100], Потери: 2.6027
Эпоха [28/100], Потери: 2.6014
Эпоха [29/100], Потери: 2.5948
Эпоха [30/100], Потери: 2.5915
Эпоха [31/100], Потери: 2.5559
Эпоха [32/100], Потери: 2.5454
Эпоха [33/100], П

In [65]:
# start_phrase = "Онегин сказал: "
# generated_text = generate_text(model, start_phrase, generate_length=100)
# print(generated_text)

In [66]:
def generate_sample(char_rnn, seed_phrase=None, max_length=200, temperature=1.0, device=device):
    '''
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs,
                        smaller temperature converges to the single most likely output
    '''
    
    if seed_phrase is not None:
        x_sequence = [token_to_idx['<sos>']] + [token_to_idx[token] for token in seed_phrase]
    else: 
        x_sequence = [token_to_idx['<sos>']]

    x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)
    
    #feed the seed phrase, if any
            
    # your code here
    # Инициализация скрытого состояния
    hidden = char_rnn.init_hidden(x_sequence.size(0)).to(device)
    char_rnn.eval()  # Переключение в режим оценки

    # Прогоняем начальную последовательность через модель
    with torch.no_grad():
        for i in range(max_length):
            output, hidden = char_rnn(x_sequence[:, -1].view(1, -1), hidden)  # Берем последний символ
            output_dist = output.view(-1).div(temperature).exp()  # Применяем температуру
            top_i = torch.multinomial(output_dist, 1)[0]  # Выбор следующего символа
            
            x_sequence = torch.cat((x_sequence, top_i.view(1, 1)), dim=1)  # Добавляем новый символ к последовательности


    
    return ''.join([tokens[ix] for ix in x_sequence.cpu().data.numpy()[0]])

Пример текста сгенерированного обученной моделью доступен ниже. Не страшно, что в тексте много несуществующих слов. Используемая модель очень проста: это простая классическая RNN.

In [68]:
print(generate_sample(model, ' мой дядя самых честных правил', max_length=500, temperature=0.8))

<sos> мой дядя самых честных правилюбы
са кристо о ведный ране милачит плестые сзнежно е
да крылась узарых,
немил наеть медной,
и мни тае всё ость овадежна, вопот наворлит, наздой никам;


xxiiii

на поторьнай поэная петтый своть к подриго.
сниша полки,
черь.
он но в скуглянь овото нена насоны что безвом
и квола…




xxvi

не быльные хонимы
и вы на на дорой
поакнодов нам глеталися в онать на стади в ракот.
о породил ей жерди стракиветь.
отереских звой свренн.



xviv

но тновит, пернетсчей солубила.
на двучев не, радомной изазьян


### Сдача задания
Сгенерируйте десять последовательностей длиной 500, используя строку ' мой дядя самых честных правил'. Температуру для генерации выберите самостоятельно на основании визуального качества генериуремого текста. Не забудьте удалить все технические токены в случае их наличия.

Сгенерированную последовательность сохрание в переменную `generated_phrase` и сдайте сгенерированный ниже файл в контест.

In [69]:
seed_phrase = ' мой дядя самых честных правил'

In [ ]:
# generated_phrases = # your code here

generated_phrases = [
    generate_sample(
        model,
        ' мой дядя самых честных правил',
        max_length=470,
        temperature=1.
    ).replace('<sos>', '')
    for _ in range(10)
]

In [77]:
print([len(generated_phrases[i]) for i in range(10)])

[500, 500, 500, 500, 500, 500, 500, 500, 500, 500]


In [78]:
# do not change the code in the block below
# __________start of block__________

import json
if 'generated_phrases' not in locals():
    raise ValueError("Please, save generated phrases to `generated_phrases` variable")

for phrase in generated_phrases:

    if not isinstance(phrase, str):
        raise ValueError("The generated phrase should be a string")

    if len(phrase) != 500:
        raise ValueError("The `generated_phrase` length should be equal to 500")

    assert all([x in set(tokens) for x in set(list(phrase))]), 'Unknown tokens detected, check your submission!'
    

submission_dict = {
    'token_to_idx': token_to_idx,
    'generated_phrases': generated_phrases
}

with open('submission_dict.json', 'w') as iofile:
    json.dump(submission_dict, iofile)
print('File saved to `submission_dict.json`')
# __________end of block__________

File saved to `submission_dict.json`


На этом задание завершено. Поздравляем!